In [1]:
import seaborn as sns
import plotly
import matplotlib as plt
import pandas as pd
import numpy as np
import statsmodels as sm
import copy
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('/home/siarhei/Programming/ML/Data/Predict Future Sales/new_test(lof).csv')
raw_data = pd.read_csv('/home/siarhei/Programming/ML/Data/Predict Future Sales/sales_train.csv')
test = pd.read_csv('/home/siarhei/Programming/ML/Data/Predict Future Sales/test.csv')

In [4]:
def prepare_data(data, test):
    #to be able to split the data later
    data['is_train'] = 1
    test['is_train'] = 0
    test['date_block_num'] = 34
    df = pd.concat([data, test], sort=False).drop(columns=['Unnamed: 0'])
    
    data_filtered = df.query("date_block_num < 34").sort_values(by=['shop_id', 'item_id', 'date'], ascending=[True, True, False]).drop_duplicates(subset=['shop_id', 'item_id'], keep='first')
    data_filtered.set_index(['shop_id', 'item_id'], inplace=True)
    df.loc[df['date_block_num'] == 34, 'item_price'] = df.loc[df['date_block_num'] == 34].set_index(['shop_id', 'item_id']).index.map(data_filtered['item_price'])
    df_filtered = df[df['is_train'] == 0]
    df = df[df['is_train'] == 1]
    df_filtered.drop_duplicates(subset='ID', keep='last', inplace=True)
    df = pd.concat([df, df_filtered], ignore_index=True)
    
    average_price = df.groupby('item_id')['item_price'].mean()
    df['item_price'].fillna(average_price,inplace=True)
    
    mean_prices = raw_data.groupby('item_id')['item_price'].mean()
    df['item_price'] = df['item_price'].fillna(df['item_id'].map(mean_prices))

    average_df_price = data['item_price'].mean(skipna=True)
    df['item_price'].fillna(average_df_price,inplace=True)
    return df

In [5]:
df = prepare_data(data, test)

Main purposes(this exact version of notebook):
    
    1. Add LAGs features(from EDA) without data leakage
    2. Train the base model(LR) to check the correct prediction on the received data
    
*To save your precious time, the logic of adding each lag feature was described (in the comments before every function)

In [6]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [7]:
class ModelValidator:
    def __init__(self, data, target_name):
        
        #Saving the created model
        self.model = None
        
        #Get only target data
        self.target_data = data[[target_name]]
        self.target_name = target_name
        
        #Get data without target column
        self.data = data.drop(target_name, axis = 1)
        
        
        self.train_x = None
        self.train_y = None
        
        self.test_x = None
        self.test_y = None
        

    def get_model(self):
        return self.model
    
    def get_test(self):
        return self.test_x, self.test_y
    
    def get_train(self):
        return self.train_x, self.train_y

    #*will be modified in DS-2.2
    #use here to check if the data is relevant for checking on kaggle
    def set_model(self, model_name):
        if model_name == 'lr':
            self.model = Lasso(alpha=0.3)
    
    def train_model(self):
        for i in range(len(self.train_x)):
            self.model.fit(self.train_x[i], self.train_y[i])
            y_pred = self.model.predict(self.test_x[i])
            print(f'Batch : {i}\nMSE : {mean_squared_error(self.test_y[i], y_pred)}\nMAE : {mean_absolute_error(self.test_y[i], y_pred)}\n R2 : {r2_score(self.test_y[i], y_pred)}')
    
    
    def expanding_windows(self, num_of_windows=33):
        splited_by_month_x = []
        splited_by_month_y = []
        
        data_blocks = self.data['date_block_num'].unique()
        
        #divide X and Y by months (date_block_num)
        for month in data_blocks:
            df = self.data[self.data['date_block_num'].isin([month])]
            indices = self.data.index[self.data['date_block_num'] == month]
            
            splited_by_month_x.append(df.values.tolist()) #3D array [DATA[MONTH[month, shop, id, ..., ...]]]
            splited_by_month_y.append(self.target_data.loc[indices].values.tolist())
        
        self.train_x = [splited_by_month_x[0]]
        self.train_y = [splited_by_month_y[0]]
        
        self.test_x = [splited_by_month_x[1]]
        self.test_y = [splited_by_month_y[1]]
        
        for i in range(1, num_of_windows):
            self.train_x.append(self.train_x[-1]+splited_by_month_x[i])
            self.train_y.append(self.train_y[-1]+splited_by_month_y[i])

            self.test_x.append(splited_by_month_x[i+1])
            self.test_y.append(splited_by_month_y[i+1])
        
        #return splited_by_month_x, splited_by_month_y

In [8]:
class Feature_extraction:
    def __init__(self, data):
        self.data = data
        
    #this function is creating a new column 'item_age' 
    #which represents the difference between 
    #the date_block_num of last sell for this item_id 
    #and 
    #the date_block_num of current item_id
    def add_global_item_age(self):
        matrix = copy.deepcopy(self.data)    
        oldest_date_year_month = matrix.groupby('item_id')['date_block_num'].min()
        merged_df = pd.merge(matrix, oldest_date_year_month, on='item_id')
        merged_df = merged_df.rename(columns={'date_block_num_y': 'oldest_date'})
        merged_df['item_age'] = merged_df['date_block_num_x'] - merged_df['oldest_date']
        merged_df.drop(columns=['oldest_date'], inplace=True)
        self.data = copy.deepcopy(merged_df)
        self.data.rename(columns={'date_block_num_x':'date_block_num'}, inplace=True)

    #this function adds an interval between product sales
    #if there were no sales before, the value of the interval is set by the variable 'value' default = 6666.0
    def add_sales_interval(self, fill_na=True, value = 6666.0):
        matrix = copy.deepcopy(self.data)
        matrix['date_block_num_diff'] = matrix.groupby('item_id')['date_block_num'].diff().apply(lambda x: x-1 if x == 1 else x)
        if fill_na:
            idx = matrix['date_block_num_diff'].isnull( )
            matrix['date_block_num_diff'][ idx ] = value
        self.data = copy.deepcopy(matrix)
    
    #this function adds average sales from previous months(not including this month to avoid data leakage)
    #for first month average_prev_sales = variable 'nan_values' default = 0.0 (because there is no sales history)
    # example 
    # date_block_num = 0 ; item_id = 10 ; item_cnt_day = 1.0 -> average_prev_sales = 0.0 (first month of selling)
    # date_block_num = 1 ; item_id = 10 ; item_cnt_day = 9.0 -> average_prev_sales = 1.0 (1.0/1 = 1.0)
    # date_block_num = 2 ; item_id = 10 ; item_cnt_day = 5.0 -> average_prev_sales = 5.0 ((1+9)/2 = 5.0)
    def add_avg_sales(self, nan_values=0.0):
        df = copy.deepcopy(self.data)
        df['average_prev_sales'] = np.nan
        date_block_nums = df['date_block_num'].unique()

        for date_block_num in date_block_nums:
            if date_block_num == 0:
                prev_sales = df[df['date_block_num'] == date_block_num]
                prev_sales = prev_sales.groupby('item_id')['item_cnt_day'].sum()/(date_block_num+1)
                df.loc[df['date_block_num'] == date_block_num,'average_prev_sales'] = nan_values
                continue
            prev_sales = df[df['date_block_num'] < date_block_num]
            prev_sales = prev_sales.groupby('item_id')['item_cnt_day'].sum()/(date_block_num)
            df.loc[df['date_block_num'] == date_block_num,'average_prev_sales'] = df.loc[df['date_block_num'] == date_block_num,'item_id'].map(prev_sales)
        df.fillna(nan_values, inplace=True)
        self.data = copy.deepcopy(df)
    
    #this function adds months from first sell in shop
    def add_shop_age(self):
        matrix = copy.deepcopy(self.data)
        min_date_block_num = matrix.groupby('shop_id')['date_block_num'].min()
        matrix = pd.merge(matrix, min_date_block_num, on='shop_id', how='left', suffixes=('', '_min'))
        matrix['shop_age_in_months'] = matrix['date_block_num'] - matrix['date_block_num_min']
        matrix.drop(columns=['date_block_num_min'], inplace=True)
        self.data = copy.deepcopy(matrix)
    
    #this function adds interval(in months) between the current sale and the last sale of this shop
    #if there is no previous sale, the value is set according to the variable 'value' default = 9999.9
    def add_store_interval(self, fill_na=True, value = 9999.9):
        matrix = copy.deepcopy(self.data)
        matrix = matrix.sort_values(by=['shop_id','date_block_num','date'])
        matrix['month_from_prev_sale'] = matrix.groupby('shop_id')['date_block_num'].diff()
        
        if fill_na:
            idx = matrix['month_from_prev_sale'].isnull( )
            matrix['month_from_prev_sale'][ idx ] = value
        self.data = copy.deepcopy(matrix)
    
    #just add a target column(item_cnt_month)
    def add_monthly_sales(self):
        matrix = copy.deepcopy(self.data)
        matrix['monthly_sales'] = matrix.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].transform('sum')
        self.data = copy.deepcopy(matrix)
    
    def sort_data(self, column_name):
        self.data.sort_values(by=column_name, inplace=True)
    
    def get_data(self):
        return self.data

In [10]:
a = copy.deepcopy(df)

In [11]:
FE = Feature_extraction(a)

In [12]:
FE.sort_data('date_block_num')
FE.add_sales_interval()
FE.add_monthly_sales()
FE.add_shop_age()
FE.add_store_interval()
FE.add_avg_sales()
FE.add_global_item_age()
FE.sort_data('date_block_num')
df = FE.get_data()

In [13]:
df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,is_train,ID,date_block_num_diff,monthly_sales,shop_age_in_months,month_from_prev_sale,average_prev_sales,item_age
0,01.01.2013,0,2,20798,999.000000,1.0,1,0.0,0.0,1.0,0,9999.9,0.000000,0
529299,29.01.2013,0,45,14754,809.000000,1.0,1,0.0,0.0,1.0,0,0.0,0.000000,0
529306,29.01.2013,0,46,14754,899.000000,1.0,1,0.0,0.0,2.0,0,0.0,0.000000,0
529307,30.01.2013,0,46,14754,809.000000,1.0,1,0.0,0.0,2.0,0,0.0,0.000000,0
116686,05.01.2013,0,23,3686,899.000000,1.0,1,0.0,0.0,1.0,0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524354,0.0,34,12,20912,857.347333,0.0,0,39713.0,0.0,0.0,34,0.0,0.441176,5
2524353,0.0,34,10,20912,857.347333,0.0,0,34613.0,0.0,0.0,34,0.0,0.441176,5
2524352,0.0,34,7,20912,857.347333,0.0,0,29513.0,0.0,0.0,34,0.0,0.441176,5
1847226,0.0,34,47,13903,199.000000,0.0,0,128748.0,0.0,0.0,34,0.0,12.176471,34


In [14]:
test = df[df.is_train == False]
train = df[df['is_train']==True]

In [15]:
train.drop(columns=['date','item_cnt_day', 'is_train', 'ID'], inplace=True)
test.drop(columns=['date','item_cnt_day', 'is_train', 'monthly_sales'], inplace=True)

In [16]:
mv = ModelValidator(data=train, target_name='monthly_sales')
mv.expanding_windows()

In [17]:
mv.set_model('lr')
mv.train_model()

Batch : 0
MSE : 85.36269936254912
MAE : 5.095543675096315
 R2 : 0.0074249769891538975
Batch : 1
MSE : 155.88292106054368
MAE : 4.61828028340388
 R2 : 0.05682576017049479
Batch : 2
MSE : 69.30739046976181
MAE : 4.404233486677269
 R2 : 0.1087794769475674
Batch : 3
MSE : 335.91593677683915
MAE : 5.429926595992062
 R2 : 0.054401684300145026
Batch : 4
MSE : 966.4067606456057
MAE : 7.4553903152084136
 R2 : 0.081147963824316
Batch : 5
MSE : 538.6657200655625
MAE : 6.561758223496567
 R2 : 0.3073796900153425
Batch : 6
MSE : 502.0131502594688
MAE : 6.865536467319049
 R2 : 0.40430205909408967
Batch : 7
MSE : 854.9044999299342
MAE : 8.866829748520946
 R2 : 0.3514958626046534
Batch : 8
MSE : 658.0411640724185
MAE : 9.493304680922895
 R2 : 0.42475375054353925
Batch : 9
MSE : 759.7533456019013
MAE : 9.86292856328456
 R2 : 0.393083208870593
Batch : 10
MSE : 842.9879993764698
MAE : 9.060605667802617
 R2 : 0.46191984250155926
Batch : 11
MSE : 628.337415304738
MAE : 8.564833108516156
 R2 : 0.247922159592

In [18]:
model = mv.get_model()

In [19]:
test.sort_values(by=['ID'],inplace=True)

In [20]:
pred = model.predict(test.drop('ID', axis=1).values.tolist())

In [21]:
subm = test.drop(['shop_id','item_id', 'date_block_num', 'item_price', 'date_block_num_diff','shop_age_in_months','month_from_prev_sale','average_prev_sales','item_age'], axis=1)
subm['item_cnt_month'] = pred
subm.ID = subm.ID.astype('int32')
subm.to_csv('subm.csv',index=False)

The resulting prediction gave me Score: 6.24153 on Kaggle

Of course this is definitely not a good score

but it shows that the data is processed correctly and the necessary structure is saved for Submission

In [22]:
#x = [[[0, 0, 0],[0, 0, 0]], 
#     [[1, 1, 1],[1, 1, 1]], 
#     [[2, 2, 2],[2, 2, 2]],
#     [[3, 3, 3],[3, 3, 3]],
#     [[4, 4, 4],[4, 4, 4]],
#     [[5, 5, 5],[5, 5, 5]]]
#
#y = [[[0.0],[0.0]],
#     [[1.0],[1.0]],
#     [[2.0],[2.0]],
#     [[3.0],[3.0]],
#     [[4.0],[4.0]],
#     [[5.0],[5.0]]]
#
#train_x = [x[0]]
#train_y = [y[0]]
#
#test_x = [x[1]]
#test_y = [y[1]]
#
#for i in range(1, len(x)-1):
#    train_x.append(train_x[-1]+x[i])
#    train_y.append(train_y[-1]+y[i])
#    
#    test_x.append(x[i+1])
#    test_y.append(y[i+1])
#
#
#for i in range(len(train_x)):
#    print('--------------------------')
#    print(f'TRAIN X : {train_x[i]}\n')
#    print(f'TRAIN Y : {train_y[i]}\n')
#    print(f'TEST X : {test_x[i]}\n')
#    print(f'TEST Y :{test_y[i]}\n')